In [3]:
%run utils.py

In [ ]:
#util.py
#from langchain.llms import OpenAI
#The above is no longer avialable, so replaced it with the below import :)
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.tools import DuckDuckGoSearchRun

# Function to generate video script
def generate_script(prompt,video_length,creativity,api_key):
    
    # Template for generating 'Title'
    title_template = PromptTemplate(
        input_variables = ['subject'], 
        template='Please come up with a title for a YouTube video on the  {subject}.'
        )

    # Template for generating 'Video Script' using search engine
    script_template = PromptTemplate(
        input_variables = ['title', 'DuckDuckGo_Search','duration'], 
        template='Create a script for a YouTube video based on this title for me. TITLE: {title} of duration: {duration} minutes using this search data {DuckDuckGo_Search} '
    )

    #Setting up OpenAI LLM
    llm = ChatOpenAI(temperature=creativity,openai_api_key=api_key,
            model_name='gpt-3.5-turbo') 
    
    #Creating chain for 'Title' & 'Video Script'
    title_chain = LLMChain(llm=llm, prompt=title_template, verbose=True)
    script_chain = LLMChain(llm=llm, prompt=script_template, verbose=True)

    
    # https://python.langchain.com/docs/modules/agents/tools/integrations/ddg
    search = DuckDuckGoSearchRun()

    # Executing the chains we created for 'Title'
    title = title_chain.invoke(prompt)

    # Executing the chains we created for 'Video Script' by taking help of search engine 'DuckDuckGo'
    search_result = search.run(prompt) 
    script = script_chain.run(title=title, DuckDuckGo_Search=search_result,duration=video_length)

     # Returning the output
    return search_result,title,script



In [4]:
#app.py
import streamlit as st 
from utils import generate_script

# Applying Styling
st.markdown("""
<style>
div.stButton > button:first-child {
    background-color: #0099ff;
    color:#ffffff;
}
div.stButton > button:hover {
    background-color: #00ff00;
    color:#FFFFFF;
    }
</style>""", unsafe_allow_html=True)


# Creating Session State Variable
if 'API_Key' not in st.session_state:
    st.session_state['API_Key'] =''


st.title('❤️ YouTube Script Writing Tool') 

# Sidebar to capture the OpenAi API key
st.sidebar.title("😎🗝️")
st.session_state['API_Key']= st.sidebar.text_input("What's your API key?",type="password")
st.sidebar.image('./Youtube.jpg',width=300, use_column_width=True)


# Captures User Inputs
prompt = st.text_input('Please provide the topic of the video',key="prompt")  # The box for the text prompt
video_length = st.text_input('Expected Video Length 🕒 (in minutes)',key="video_length")  # The box for the text prompt
creativity = st.slider('Creativity limit ✨ - (0 LOW || 1 HIGH)', 0.0, 1.0, 0.2,step=0.1)

submit = st.button("Generate Script for me")


if submit:
    
    if st.session_state['API_Key']:
        search_result,title,script = generate_script(prompt,video_length,creativity,st.session_state['API_Key'])
        #Let's generate the script
        st.success('Hope you like this script ❤️')

        #Display Title
        st.subheader("Title:🔥")
        st.write(title)

        #Display Video Script
        st.subheader("Your Video Script:📝")
        st.write(script)

        #Display Search Engine Result
        st.subheader("Check Out - DuckDuckGo Search:🔍")
        with st.expander('Show me 👀'): 
            st.info(search_result)
    else:
        st.error("Ooopssss!!! Please provide API key.....")

2024-04-24 13:46:08.741 
  command:

    streamlit run C:\Users\admin\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-04-24 13:46:08.757 Session state does not function when running a script without `streamlit run`


In [13]:
#utils.py
#Import necessary modules and classes
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.tools import DuckDuckGoSearchRun


#generate script function
def generate_script(prompt,video_length,creativity,api_key):
    
    #template
    title_template = PromptTemplate(
        input_variables = ['subject'],
        template = 'Please come up with a title for a YouTube video on the {subject}.'
        ) 
    script_template = PromptTemplate(
        input_variables = ['title','DuckDuckGo_Search','duration'],
        template = 'Create a script for a YouTube video based on this title for me. TITLE: {title} of duration: {duration} minutes using this search data {DuckDuckGo_Search} ')
    # model instance
    llm = ChatOpenAI(
        model_name = gpt-3.5-turbo,
        temperature = creativity,
        Openai_api_key = api_key)
    
    #chain
    title_chain = LLMChain(llm=llm,prompt=title_template, verbose = True)
    script_chain = LLMChain(llm=llm,prompt=script_template,verbose = True)
    #executing chain - title search script
    search = DuckDuckGoSearchRun()
    title = title_chain.invoke(prompt)
    search_reasult = search.run(prompt)
    script = script_chain.run(title=title,DuckDuckGoSearch=search_reasult,duration=video_length)
   
    
    #return output
    return  search_result,title,script

In [15]:
#import library
import streamlit as st
from utils import generate_script

#styling
st.markdown("""
    <style>
    div.stButton > button:first_child{
    background_color:#0099ff,
    color:#ffffff;
    }
    div.stButton > button:hover{
    background_color:#00ff00,
    color:#FFFFFF;
    }
    </style>""")

#session variables
if 'API_Key' not in st.session_state:
    st.session_state['API_Key'] = ''
    
#title,sidebartile,session_state, sidebar
st.title('❤️ YouTube Script Writing Tool')
st.sidebar.title("😎🗝️")
st.session_state['API_Key'] = st.sidebar.text_input("What's your API key?",type="password")
st.sidebar.image('./Youtube.jpg',width=300, use_column_width=True)

#user input: prompt,video_length,creativity
prompt = st.text_input('Please provide the topic of the video',key="prompt")
video_length = st.text_input('Expected Video Length 🕒 (in minutes)',key="video_length")
creativity = st.slider('Creativity limit ✨ - (0 LOW || 1 HIGH)', 0.0, 1.0, 0.2,step=0.1)

#submit
submit = st.button("Generate script for me")
#once submit, invoke generate script function 
if submit:
    if st.sessiom_state['API_Key']:
        search_result,title,script = generate_script(prompt,video_length,creativity,st.session_state['API_Key'])
        st.success('Hope you like this script ❤️')

        st.subheader("Title:🔥")
        st.write(title)

        st.subhear("Your Video Script:📝")
        st.write(script)

        st.subheader("Check Out - DuckDuckGo Search:🔍")
        with st.expander('Show me 👀'): 
            st.info(search_result)

    else:
        st.error("Ooopssss!!! Please provide API key.....")                                             

2024-04-24 23:49:27.948 
  command:

    streamlit run C:\Users\admin\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-04-24 23:49:27.948 Session state does not function when running a script without `streamlit run`
